# CUDA编程模型---CUDA存储单元的使用

#### 通过之前的学习，我们已经初步掌握了利用GPU加速应用程序的方法。接下来，我们针对更多细节加以训练。本次实验课将会介绍：  
1. Cuda编程模型中多种类型的存储单元
2. 利用共享存储单元来加速矩阵乘法


#### Cuda编程模型中的多种存储单元

![gpu_memory](gpu_memory.png)

- 寄存器

寄存器是速度最快的存储单元，位于GPU芯片的SM上，用于存储局部变量。每个SM（SMX）上有成千上万的32位寄存器，当kernel函数启动后，这些寄存器被分配给指定的线程来使用。由于不同kernel函数需要的寄存器数量也不相等，所以，也有一个规定一个线程的最大寄存器数量是256个。寄存器的最小单位是register file，所以，在很多图上也会用register file来表示。

- Local Memory

Local Memory本身在硬件中没有特定的存储单元，而是从Global Memory虚拟出来的地址空间。Local Memory是为寄存器无法满足存储需求的情况而设计的，主要是用于存放单线程的大型数组和变量。Local Memory是线程私有的，线程之间是不可见的。由于GPU硬件单位没有Local Memory的存储单元，所以，针对它的访问是比较慢的。从上面的表格中，也可以看到跟Global Memory的访问速度是接近的。

- Shared Memory

Shared Memory位于GPU芯片上，访问延迟仅次于寄存器。Shared Memory是可以被一个Block中的所有Thread来进行访问的，可以实现Block内的线程间的低开销通信。在SMX中，L1 Cache跟Shared Memory是共享一个64KB的告诉存储单元的，他们之间的大小划分不同的GPU结构不太一样；

- Constant Memory

Constant Memory类似于Local Memory，也是没有特定的存储单元的，只是Global Memory的虚拟地址。因为它是只读的，所以简化了缓存管理，硬件无需管理复杂的回写策略。Constant Memory启动的条件是同一个warp所有的线程同时访问同样的常量数据。

- Global Memory

Global Memory在某种意义上等同于GPU显存，kernel函数通过Global Memory来读写显存。Global Memory是kernel函数输入数据和写入结果的唯一来源。

- Texture Memory

Texture Memory是GPU的重要特性之一，也是GPU编程优化的关键。Texture Memory实际上也是Global Memory的一部分，但是它有自己专用的只读cache。这个cache在浮点运算很有用，Texture Memory是针对2D空间局部性的优化策略，所以thread要获取2D数据就可以使用texture Memory来达到很高的性能。从读取性能的角度跟Constant Memory类似。

- Host Memory

主机端存储器主要是内存可以分为两类：可分页内存（Pageable）和页面 （Page-Locked 或 Pinned）内存。

可分页内存通过操作系统 API(malloc/free) 分配存储器空间，该内存是可以换页的，即内存页可以被置换到磁盘中。可分页内存是不可用使用DMA（Direct Memory Acess)来进行访问的，普通的C程序使用的内存就是这个内存。

#### 4.利用Shrared Memory来优化矩阵乘法

![shared_memory](shared_memory.png)

当我们在处理矩阵乘法时，假设矩阵M(m,k)\*N(k,n) = P(m,n)。那么，矩阵M中的一个数值m(x,y),就要被grid中所有满足threadIdx.y+blockIdx.y\*blockDim.y = y的线程从Global Memory中读一次，一共就是K次。那么，我们看到这么多重复读取，就可以把这个变量放在Shared Memory中，极大地减少每次的读取时间。我们可以按照下面的代码来修改martix_mul的核函数：

```C++   
__global__ void gpu_matrix_mult_shared(int *d_a, int *d_b, int *d_result, int m, int n, int k) 
{
    __shared__ int tile_a[BLOCK_SIZE][BLOCK_SIZE];
    __shared__ int tile_b[BLOCK_SIZE][BLOCK_SIZE];

    int row = blockIdx.y * BLOCK_SIZE + threadIdx.y;
    int col = blockIdx.x * BLOCK_SIZE + threadIdx.x;
    int tmp = 0;
    int idx;

    for (int sub = 0; sub < gridDim.x; ++sub) 
    {
        idx = row * n + sub * BLOCK_SIZE + threadIdx.x;
        tile_a[threadIdx.y][threadIdx.x] = row<n && (sub * BLOCK_SIZE + threadIdx.x)<n? d_a[idx]:0;
        idx = (sub * BLOCK_SIZE + threadIdx.y) * n + col;
        tile_b[threadIdx.y][threadIdx.x] = col<n && (sub * BLOCK_SIZE + threadIdx.y)<n? d_b[idx]:0;
        
        __syncthreads();
        for (int k = 0; k < BLOCK_SIZE; ++k) 
        {
            tmp += tile_a[threadIdx.y][k] * tile_b[k][threadIdx.x];
        }
        __syncthreads();
    }
    if(row < n && col < n)
    {
        d_result[row * n + col] = tmp;
    }
}

```

![array_2d](array_2to1.png)

修改[matrix_mul.cu](matrix_mul.cu)文件，利用Makefile编译，并执行。如果遇到困难，请参考[result4.cu](result4.cu)

In [ ]:
!make

In [ ]:
!./matrix_mul

利用nvprof来查看性能

In [ ]:
!sudo /usr/local/cuda/bin/nvprof ./matrix_mul

课后作业： 

1. 修改BLOCK_SIZE查看性能变化
2. 修改代码，尝试产生bank_conflict